In [ ]:
#Импортируем библиотеку
!pip install python-telegram-bot==13.15 transformers

In [ ]:
#Логирование
import logging

#Настройка телеграм-бота
from telegram.ext import Updater, MessageHandler, Filters

#Загрузка модели для анализа текста
from transformers import pipeline

In [ ]:
#Используем токен бота
TELEGRAM_TOKEN = '7616182006:AAGyJMQPCY8tONIk3LOlhegI1ZT0IT3sR4A'

#Настройка логирования
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)

#Загружаем мультиязычную модель суммаризации (поддерживает русский язык)
summarizer = pipeline("summarization", model="csebuetnlp/mT5_multilingual_XLSum")

#Обработчик команды /start
def start(update, context):
    update.message.reply_text(
        "👋 Привет! Я бот для суммаризации текста.\n\n"
        "Отправьте мне длинный текст (от 100 слов), и я сделаю его краткое содержание."
    )

#Обработчик текстовых сообщений
def handle_message(update, context):
    user_text = update.message.text

    if len(user_text.split()) < 50:
        update.message.reply_text("📏 Пожалуйста, пришлите более длинный текст (хотя бы 50–100 слов).")
        return

    #Сообщаем пользователю, что началась обработка
    processing_message = update.message.reply_text("⏳ Обрабатываю текст...")

    try:
        summary = summarizer(user_text, max_length=256, min_length=64, do_sample=False)
        result = summary[0]['summary_text']
        processing_message.edit_text(f"📌 Готово! Краткое содержание:\n{result}")

    except Exception as e:
        logging.error(f"Ошибка при суммаризации: {e}")
        processing_message.edit_text("⚠️ Произошла ошибка при обработке текста. Попробуйте позже.")



config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cpu


In [ ]:
#Инициализация и запуск бота
def main():
    updater = Updater(token=TELEGRAM_TOKEN, use_context=True)
    dispatcher = updater.dispatcher

    #Обработка текстовых сообщений (не команд)
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, handle_message))

    print("Бот запущен! Ожидаю сообщения...")
    updater.start_polling()
    updater.idle()

if __name__ == '__main__':
    main()

🤖 Бот запущен! Ожидаю сообщения...
